## 1.0 - Basic LLM Prompting with vLLM and Langchain 

## Preparation

In [ ]:
!pip install -q langgraph==0.2.35 langchain_experimental==0.0.65 langchain-openai==0.1.25 termcolor==2.3.0 duckduckgo_search==7.1.0 openapi-python-client==0.12.3 langchain_community==0.2.19 wikipedia==1.4.0

In [2]:
# Imports
import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [3]:
INFERENCE_SERVER_URL = os.getenv('API_URL')
MODEL_NAME = "mistral-7b-instruct"
API_KEY= os.getenv('API_KEY')

### Langchain

Langchain (https://www.langchain.com/) is a framework for developing applications powered by language models. It will take care for us of all the boilerplate code we would have to manually write to properly query an LLM.

We will start by creating an **llm** instance, defined by the location where the LLM API can be queried and some parameters that will be applied to the model. For example, `max_new_tokens` will instruct the model to answer with a maximum of 512 tokens (words or parts of words). `temperature`, set really low here, will instruct the model to stay truth-grounded, and not try to be too "creative". After all, we're not trying to write a fancy poem here!

#### Create the LLM instance

In [4]:
# LLM definition
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

#### Create the Prompt

In [5]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

## First Query to LLMs

#### Create the Chain using the different components

In [6]:
conversation = LLMChain(llm=llm,
                            prompt=PROMPT,
                            verbose=False,
                            )

/tmp/ipykernel_2125/595551351.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  conversation = LLMChain(llm=llm,


#### Let's talk...

In [7]:
first_input = "Describe what is the S&P500 in 100 words or less."
conversation.predict(input=first_input);

 The S&P 500, or Standard & Poor's 500, is a stock market index that measures the performance of 500 large companies listed on stock exchanges in the United States. It is widely regarded as a primary gauge of U.S. equities and is often used to reflect the overall health of the American economy. Companies included in the S&P 500 are selected based on market size, liquidity, and industry group representation.

#### LLM without Conversational Memory

In [10]:
second_input = "How many companies are included?"
conversation.predict(input=second_input);

 I don't have real-time data or the ability to browse the internet, so I can't provide an exact number of companies that currently exist. However, it's estimated that there are millions of active businesses worldwide. The number is constantly changing due to new businesses being formed and others closing.

## What's Next?

Let's head over to give our LLM Conversational Memory!